## Getting a bit more deeper into Agents

### Conversational ReAct Agents
This agent is designed for use in conversational settings. It incorporates the React framework to determine which tool to use and utilizes memory to remember previous conversation interactions.

In [19]:
from langchain.agents import create_react_agent, load_tools
from langchain_openai import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.agents import initialize_agent
from langchain_openai import OpenAI
from dotenv import load_dotenv
load_dotenv() 

True

In [9]:
llm = ChatOpenAI(temperature=0, model_name='gpt-3.5-turbo-0125')

In [10]:
tools = load_tools(["llm-math"],  llm=llm)

In [11]:
memory = ConversationBufferMemory(memory_key="chat_history")

conversational_agent = initialize_agent(
    agent="conversational-react-description", 
    tools=tools, 
    llm=llm,
    verbose=True,
    max_iterations=3,
    memory=memory,)

C:\Users\rakes\anaconda3\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [12]:
output_1=conversational_agent.run("Add 7 to 9 and tell me the result")
output_2=conversational_agent.run("Add 5 to the previous result")

C:\Users\rakes\anaconda3\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: Calculator
Action Input: Add 7 to 9
Observation: Answer: 16
Thought:Do I need to use a tool? No
AI: The result of adding 7 to 9 is 16.

> Finished chain.


> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: Calculator
Action Input: Add 16 to 5
Observation: Answer: 21
Thought:Do I need to use a tool? No
AI: The result of adding 5 to the previous result of 16 is 21.

> Finished chain.


In [13]:
print(output_1)
print(output_2)

The result of adding 7 to 9 is 16.
The result of adding 5 to the previous result of 16 is 21.


## ReAct Docstore
This agent utilizes the React framework to communicate with a docstore. It requires the availability of a Search tool and a Lookup tool with identical names. The Search tool is used to search for a document, while the Lookup tool looks up a term within the most recently found document.

A `docstore` in Langchain is essentially a storage system for documents. It provides a way for Langchain agents to access and interact with information. Think of it like a library, but instead of books, it holds pieces of text and associated data.

In [16]:
# !pip install Wikipedia

  Created wheel for Wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11689 sha256=be8e3e8c4c7c50518af1625591dddb971922160bd5f4a038d71ce43cbd84faf9
  Stored in directory: c:\users\rakes\appdata\local\pip\cache\wheels\07\93\05\72c05349177dca2e0ba31a33ba4f7907606f7ddef303517c6a
Successfully built Wikipedia


In [20]:
from langchain.agents import initialize_agent, Tool
from langchain import Wikipedia
from langchain.agents.react.base import DocstoreExplorer

In [21]:
docstore = DocstoreExplorer(Wikipedia())
tools=[
    Tool(name="Search", func=docstore.search, description="useful for when you need to search wikipedia"),
    Tool(name="Lookup", func=docstore.lookup, description="useful for when you need to lookup a term in wikipedia")]

#intializing the agent
llm = OpenAI(temperature=0)
docstore_agent=initialize_agent(
    tools,
    llm,
    agent="react-docstore",
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=8
)

In [22]:
docstore_agent("Tell me a few key things about Elon Musk.")

C:\Users\rakes\anaconda3\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
Thought: I need to search Elon Musk and find some key information about him.
Action: Search[Elon Musk]
Observation: Could not find [Elon Musk]. Similar: ['Elon Musk', 'Twitter under Elon Musk', 'Musk family', 'Acquisition of Twitter by Elon Musk', 'Wealth of Elon Musk', 'Views of Elon Musk', 'Elon Musk (disambiguation)', 'Elon Musk vs. Mark Zuckerberg', "Elon Musk's Tesla Roadster", 'Maye Musk']
Thought: I need to be more specific and search for "Elon Musk entrepreneur" to find information about his career.
Action: Search[Elon Musk entrepreneur]
Observation: The Musk family is a wealthy family of South African origin that is largely active in the United States and Canada. The Musks are of English, Anglo-Canadian, Pennsylvania Dutch, and Swiss descent. The family is known for its entrepreneurial endeavours. Elon Musk was formerly the wealthiest person in the world, with an estimated net worth of US$232 billion as of December 2023, according to the

{'input': 'Tell me a few key things about Elon Musk.',
 'output': 'Elon Musk is an entrepreneur from a wealthy family and was the wealthiest person in the world in 2023. He is the founder of companies such as Tesla and SpaceX.'}

## Self-ask with Search
This agent utilizes the Intermediate Answer tool for self-asking questions.

In [29]:
from langchain.tools import DuckDuckGoSearchRun
search = DuckDuckGoSearchRun()

tools = [
    Tool(
        func=search.run,
        name="Intermediate Answer",
        description="useful for when you need to search the internet for information"
    )

]

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent="self-ask-with-search",
    verbose=True,
    handle_parsing_errors=True
)

In [31]:
print(agent.invoke("Question: Who was the president of US when superconductor was discovered?"))



> Entering new AgentExecutor chain...
 Yes.
Follow up: When was the superconductor discovered?
Intermediate answer: No good DuckDuckGo Search Result was found
Follow up: Who discovered the superconductor?
Intermediate answer: No good DuckDuckGo Search Result was found
So the final answer is: No information available.

> Finished chain.
{'input': 'Question: Who was the president of US when superconductor was discovered?', 'output': 'No information available.'}


The above code imports necessary modules from the LangChain library, including agents and language models. It sets up a conversational agent with a specific agent configuration called “self-ask-with-search”. The agent utilizes the “Intermediate Answer” tool, which performs Wikipedia searches.